In [237]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hvplot.pandas
from pathlib import Path

In [238]:
world_stock =pd.read_csv(Path(r"C:\Users\5th gen\Dropbox\PC\Desktop\FinTech & Machine Learning Bootcamp\Project\08-Project-1\World-Stock-Prices-Dataset.csv"))

In [239]:
# Extract data only for USA data
world_stock_df = world_stock[world_stock['Country']=='usa']

# Set Date as index
world_stock_df.set_index('Date', inplace=True)

# display results
world_stock_df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Brand_Name,Ticker,Industry_Tag,Country
Date,,,,,,,,,,,
2023-09-20 00:00:00-04:00,4.840000,4.910000,4.630000,4.670000,7441900.0,0.0,0.0,peloton,PTON,fitness,usa
2023-09-20 00:00:00-04:00,397.049988,397.989990,386.119995,386.299988,3866600.0,0.0,0.0,netflix,NFLX,entertainment,usa
2023-09-20 00:00:00-04:00,564.349976,569.219971,562.659973,563.830017,1311500.0,0.0,0.0,costco,COST,retail,usa
2023-09-20 00:00:00-04:00,138.550003,139.369995,135.199997,135.289993,46263700.0,0.0,0.0,amazon,AMZN,e-commerce,usa
2023-09-20 00:00:00-04:00,179.259995,179.699997,175.399994,175.490005,58436200.0,0.0,0.0,apple,AAPL,technology,usa


In [240]:
# The columns needed for short-term portfolio analysis
columns_for_shortterm_analysis = ['Close', 'Volume', 'Ticker']

# Select and keep only the desired columns
world_stock_df = world_stock_df[columns_for_shortterm_analysis]

# Display results
world_stock_df.head()

,Close,Volume,Ticker
Date,,,
2023-09-20 00:00:00-04:00,4.670000,7441900.0,PTON
2023-09-20 00:00:00-04:00,386.299988,3866600.0,NFLX
2023-09-20 00:00:00-04:00,563.830017,1311500.0,COST
2023-09-20 00:00:00-04:00,135.289993,46263700.0,AMZN
2023-09-20 00:00:00-04:00,175.490005,58436200.0,AAPL


In [241]:
# Change data column format is the name of the indexed date column
world_stock_df.index = world_stock_df.index.str.split().str[0]

# Display results
world_stock_df.head()

,Close,Volume,Ticker
Date,,,
2023-09-20,4.670000,7441900.0,PTON
2023-09-20,386.299988,3866600.0,NFLX
2023-09-20,563.830017,1311500.0,COST
2023-09-20,135.289993,46263700.0,AMZN
2023-09-20,175.490005,58436200.0,AAPL


In [242]:
# Calculate the daily change in volume
world_stock_df['Daily Volume Change'] = world_stock_df.groupby('Ticker')['Volume'].diff()

In [243]:
world_stock_df.dropna()

,Close,Volume,Ticker,Daily Volume Change
Date,,,,
2023-09-19,4.800000,12741900.0,PTON,5300000.0
2023-09-19,396.200012,4076800.0,NFLX,210200.0
2023-09-19,564.349976,1144700.0,COST,-166800.0
2023-09-19,137.630005,61482500.0,AMZN,15218800.0
2023-09-19,179.070007,51826900.0,AAPL,-6609300.0
...,...,...,...,...
2023-08-29,18.320000,5949600.0,FL,-2013300.0
2023-08-30,18.549999,5829500.0,FL,-120100.0
2023-08-31,19.620001,6316100.0,FL,486600.0


In [245]:
# Calculate daily returns for each ticker based on closing price
world_stock_df['Daily Return'] = world_stock_df.groupby('Ticker')['Close'].pct_change()

# Drop rows with NaN (first day of each ticker)
world_stock_df.dropna(subset=['Daily Return'], inplace=True)

# Drop rows with Daily Return equal to 0
world_stock_df = world_stock_df[world_stock_df['Daily Return'] != 0]

# Print the DataFrame with daily returns
daily_return = world_stock_df[['Ticker', 'Close', 'Volume','Daily Return']]

# Print results
daily_return


,Ticker,Close,Volume,Daily Return
Date,,,,
2023-09-18,PTON,4.630000,12031500.0,-0.035417
2023-09-18,NFLX,394.399994,4704700.0,-0.004543
2023-09-18,COST,562.719971,1003100.0,-0.002888
2023-09-18,AMZN,139.979996,42823500.0,0.017075
2023-09-18,AAPL,177.970001,67257600.0,-0.006143
...,...,...,...,...
2023-08-29,FL,18.320000,5949600.0,-0.010799
2023-08-30,FL,18.549999,5829500.0,0.012555
2023-08-31,FL,19.620001,6316100.0,0.057682


In [246]:
# List of moving average periods to calculate
ma_periods = [7, 21, 40]

# Calculate moving averages for each stock and period
def calculate_moving_averages(world_stock_df, ma_periods):
    for period in ma_periods:
        world_stock_df[f'{period}-Day MA'] = world_stock_df['Close'].rolling(window=period).mean()
    return world_stock_df

# Calculate moving averages for your DataFrame
ma_world_stock_df = world_stock_df.groupby('Ticker', group_keys=False).apply(calculate_moving_averages, ma_periods)

# Drop NaN values
ma_world_stock_df.dropna()

,Close,Volume,Ticker,Daily Volume Change,Daily Return,7-Day MA,21-Day MA,40-Day MA
Date,,,,,,,,
2019-09-26,53.466011,75334000.0,AAPL,-12279600.0,-0.005158,53.450375,52.570989,51.192676
2019-09-26,283.809998,4270800.0,ADBE,963500.0,0.022997,278.577144,280.964764,284.744500
2019-09-26,14.696296,1977500.0,AEO,-638000.0,-0.015893,14.912763,15.160961,14.911035
2019-09-26,29.469999,40745400.0,AMD,-17489400.0,-0.002370,29.990000,30.451429,30.615750
2019-09-26,86.991997,70736000.0,AMZN,872000.0,-0.016111,89.058570,90.114285,89.975400
...,...,...,...,...,...,...,...,...
2001-02-01,5.320480,10366400.0,NKE,2958400.0,-0.009634,5.224668,5.385760,5.078798
2001-02-01,1.983575,37185600.0,NVDA,1351200.0,0.005145,1.974531,1.823048,1.658848
2001-02-01,20.197939,7553400.0,PG,-778400.0,0.029371,19.211353,19.628921,19.797826


In [247]:
# List of RSI periods to calculate
rsi_periods = [7, 21, 40]

# Calculate RSI for each ticker and period
def calculate_rsi(world_stock_df, rsi_periods):
    for period in rsi_periods:
        delta = world_stock_df['Close'].diff(1)
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
        avg_gain = gain.rolling(window=period).mean()
        avg_loss = loss.rolling(window=period).mean()
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
        world_stock_df[f'{period}-Days RSI'] = rsi
    return world_stock_df

# Calculate RSI for your DataFrame
RSI_world_stock_df = world_stock_df.groupby('Ticker', group_keys=False).apply(calculate_rsi, rsi_periods)

# Print results of RSI for 
RSI_world_stock_df.dropna()

,Close,Volume,Ticker,Daily Volume Change,Daily Return,7-Days RSI,21-Days RSI,40-Days RSI
Date,,,,,,,,
2019-09-26,53.466011,75334000.0,AAPL,-12279600.0,-0.005158,47.028700,66.910334,53.159429
2019-09-26,283.809998,4270800.0,ADBE,963500.0,0.022997,48.348340,45.859019,45.224351
2019-09-26,14.696296,1977500.0,AEO,-638000.0,-0.015893,30.434937,49.546828,45.950074
2019-09-26,29.469999,40745400.0,AMD,-17489400.0,-0.002370,22.262782,45.054938,43.046776
2019-09-26,86.991997,70736000.0,AMZN,872000.0,-0.016111,21.326986,46.736706,42.492133
...,...,...,...,...,...,...,...,...
2001-02-01,5.320480,10366400.0,NKE,2958400.0,-0.009634,59.118351,50.678031,59.543571
2001-02-01,1.983575,37185600.0,NVDA,1351200.0,0.005145,33.717138,67.163409,52.933142
2001-02-01,20.197939,7553400.0,PG,-778400.0,0.029371,71.931813,43.778600,50.311536


In [248]:
def calculate_macd(world_stock_df, short_period=12, long_period=26, signal_period=9):
    # Calculate short-term EMA
    ema_short = world_stock_df['Close'].ewm(span=short_period).mean()
    
    # Calculate long-term EMA
    ema_long = world_stock_df['Close'].ewm(span=long_period).mean()
    
    # Calculate MACD line
    world_stock_df['MACD'] = ema_short - ema_long
    
    # Calculate signal line
    world_stock_df['Signal_Line'] = world_stock_df['MACD'].ewm(span=signal_period).mean()
    
    return world_stock_df

MACD_world_stock_df = calculate_macd(world_stock_df)

# Drop NaN values
MACD_world_stock_df.dropna()

,Close,Volume,Ticker,Daily Volume Change,Daily Return,MACD,Signal_Line
Date,,,,,,,
2023-09-18,4.630000,12031500.0,PTON,-710400.0,-0.035417,0.000000,0.000000
2023-09-18,394.399994,4704700.0,NFLX,627900.0,-0.004543,8.744840,4.858244
2023-09-18,562.719971,1003100.0,COST,-141600.0,-0.002888,16.362254,9.573002
2023-09-18,139.979996,42823500.0,AMZN,-18659000.0,0.017075,4.162940,7.740325
2023-09-18,177.970001,67257600.0,AAPL,15430700.0,-0.006143,-1.041915,5.127807
...,...,...,...,...,...,...,...
2023-08-29,18.320000,5949600.0,FL,-2013300.0,-0.010799,-2.092989,-1.324206
2023-08-30,18.549999,5829500.0,FL,-120100.0,0.012555,-2.126963,-1.484758
2023-08-31,19.620001,6316100.0,FL,486600.0,0.057682,-2.043986,-1.596603


In [227]:
# Calculate the 7, 21, and 40-day rolling averages of the volume change for each ticker
vol_7days = world_stock_df.groupby('Ticker')['Daily Volume Change'].rolling(window=7).mean().dropna()
vol_21days = world_stock_df.groupby('Ticker')['Daily Volume Change'].rolling(window=21).mean().dropna()
vol_40days = world_stock_df.groupby('Ticker')['Daily Volume Change'].rolling(window=40).mean().dropna()


In [228]:
vol_7days

Ticker  Date      
AAPL    2023-09-08    1.046047e+06
        2023-09-07    8.695590e+06
        2023-09-06    2.100775e+06
        2000-01-03    6.096351e+07
        2000-01-04    6.451924e+07
                          ...     
ZM      2023-08-29   -1.081000e+05
        2023-08-30   -1.119543e+06
        2023-08-31   -9.466571e+05
        2023-09-01   -4.736143e+05
        2023-09-05    8.850000e+05
Name: Daily Volume Change, Length: 217621, dtype: float64